In [4]:
from osgeo import gdal, ogr
import numpy as np
import os

def build_forest_mask(shapefile_path, raster_template_path, output_path):
    # Charger le shapefile
    shapefile = ogr.Open(shapefile_path)
    layer_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    layer = shapefile.GetLayer()

    # Charger les informations du raster modèle
    raster_template = gdal.Open(raster_template_path)
    geo_transform = raster_template.GetGeoTransform()
    projection = raster_template.GetProjection()
    x_size = raster_template.RasterXSize
    y_size = raster_template.RasterYSize

    # Créer un raster temporaire en mémoire
    driver = gdal.GetDriverByName('GTiff')
    target_ds = driver.Create(
        output_path, x_size, y_size, 1, gdal.GDT_Byte
    )
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)

    # Initialiser le raster avec la valeur 0 (zones hors forêt)
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(0)
    band.WriteArray(np.zeros((y_size, x_size), dtype=np.uint8))

    # Appliquer un script SQL pour sélectionner les zones de forêt
    sql_query = (
        f"SELECT * FROM {layer_name} WHERE CODE_TFV NOT IN ('LA4', 'LA6', 'FO0', 'FO1')"
    )
    sql_layer = shapefile.ExecuteSQL(sql_query)

    # Créer une image rasterisée en mémoire avec les zones de forêt
    gdal.RasterizeLayer(
        target_ds, [1], sql_layer,
        burn_values=[1],
        options=["ATTRIBUTE=CODE_TFV"]
    )

    # Libérer les ressources SQL
    shapefile.ReleaseResultSet(sql_layer)

    # Fermer les fichiers
    band.FlushCache()
    band = None
    target_ds = None
    shapefile = None

if __name__ == "__main__":
    # Chemins vers les fichiers nécessaires
    shapefile_path = "../../data/project/FORMATION_VEGETALE.shp"
    raster_template_path = "../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif"
    output_path = "../results/data/img_pretraitees/masque_foret.tif"

    # Créer le masque
    build_forest_mask(shapefile_path, raster_template_path, output_path)
    print(f"Masque forêt généré : {output_path}")


Masque forêt généré : ../results/data/img_pretraitees/masque_foret.tif
